# T5-Large 와 PEFT 를 활용하여 Translation 구현하기 (Large Version)

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 T5모델을 이용하여 아주 간단한 번역 문제를 풀어보겠습니다.

활용 데이터는 human-machine date 데이터로서 아주 간단한 seqeucne-to-sequence 문제입니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(훈련소요 시간 : 1시간, 설명 : 20분)

In [1]:
# performance metric
!pip install -q sacrebleu


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from IPython.display import Markdown, display

In [3]:
import os

# 첫 번째 GPU만 사용하도록 설정 (본 실습환경 때문)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Date Dataset

Dataset 은 이미 만들어 두었고, 이를 가져온다고 가정하겠습니다.

In [5]:
!unzip human_machine_date_dataset.zip

Archive:  human_machine_date_dataset.zip
   creating: human_machine_date_dataset/.ipynb_checkpoints/
 extracting: human_machine_date_dataset/.ipynb_checkpoints/dataset_dict-checkpoint.json  
 extracting: human_machine_date_dataset/dataset_dict.json  
   creating: human_machine_date_dataset/train/
  inflating: human_machine_date_dataset/train/cache-113ce0f9cbf9302b.arrow  
  inflating: human_machine_date_dataset/train/data-00000-of-00001.arrow  
  inflating: human_machine_date_dataset/train/dataset_info.json  
  inflating: human_machine_date_dataset/train/state.json  
   creating: human_machine_date_dataset/validation/
  inflating: human_machine_date_dataset/validation/cache-924eedb5e7ecb100.arrow  
  inflating: human_machine_date_dataset/validation/data-00000-of-00001.arrow  
  inflating: human_machine_date_dataset/validation/dataset_info.json  
  inflating: human_machine_date_dataset/validation/state.json  


In [6]:
from datasets import load_from_disk

data_directory = './human_machine_date_dataset'
dataset = load_from_disk(data_directory)

print("Loaded dataset:", dataset)

Loaded dataset: DatasetDict({
    train: Dataset({
        features: ['human', 'machine'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['human', 'machine'],
        num_rows: 1000
    })
})


In [8]:
dataset['train']['human'][:10]

['12, Sep 2010',
 'DECEMBER 14 1992',
 '8 JUN 1999',
 '11.03.00',
 '6/16/79',
 '1 FEBRUARY, 1988',
 '19 jun 1991',
 '12 january, 1981',
 '30 Sep 2015',
 '8 Jul, 1996']

In [9]:
dataset['train']['machine'][:10]

['2010-09-12',
 '1992-12-14',
 '1999-06-08',
 '2000-03-11',
 '1979-06-16',
 '1988-02-01',
 '1991-06-19',
 '1981-01-12',
 '2015-09-30',
 '1996-07-08']

## Data 전처리 하기

### Model 설정하기

In [10]:
MODEL_ID = "t5-large"

In [11]:
import transformers
from transformers import AutoTokenizer    
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# fast version 인지 아닌지 확인
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [12]:
max_token_length = 64

In [13]:
#print(tokenizer.vocab)

In [14]:
s = dataset["train"]['human'][0]
t = dataset["train"]['machine'][0]

s_output = tokenizer(s)
with tokenizer.as_target_tokenizer():
    t_output = tokenizer(t)

print( s_output['input_ids']  )
print( t_output['input_ids'] )
print("-"*50)
print( tokenizer.decode( s_output['input_ids'] ))
print( tokenizer.decode( t_output['input_ids'] ))

[10440, 12219, 2735, 1]
[2735, 18, 4198, 5947, 1]
--------------------------------------------------
12, Sep 2010</s>
2010-09-12</s>


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


### Prefix 넣어주기
T5 original paper에서는 모든 Task 에 task prefix를 넣도록 하는 방법을 제안합니다. 즉, 영어에서 한국어로 번역한다면 "Translate English to Korean : "이라는 prefix term 을 입력의 가장 앞에 넣어주는 방식입니다. 

그 이후에 나온 T5의 구현체마다 미미하게 다른 방식을 씁니다. 만약 Downstream task가 '번역'만 다룬다면 굳이 필요 없을 수도 있습니다. 

본 실습에서는 Prefix도 넣어주는 방법으로 구현해봅니다.

In [15]:
max_token_length = 64

ENABLE_PREFIX = True

source_lang = "human"
target_lang = "machine"

if ENABLE_PREFIX:
    prefix = f"translate {source_lang} to {target_lang}: "
else:
    prefix = ""
    
source_lang = source_lang.lower()
target_lang = target_lang.lower()
    
def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    
    tokenized_examples = tokenizer(
                            inputs,
                            text_target=targets, 
                            max_length=max_token_length, 
                            truncation=True
                         )
    return tokenized_examples

### tokenizer test해보기

In [16]:
print(preprocess_function(dataset['train'][:2]).keys())
out = preprocess_function(dataset['train'][:2])
print(out)
print("-"*50)
print("Input Tokens  : ", out['input_ids'][0])
print("Input IDs     : ", tokenizer.decode(out['input_ids'][0]) )
print("-"*50)
print("Output Tokens : ", out['labels'][0])
print("Output IDs    : ", tokenizer.decode(out['labels'][0]) )

KeysView({'input_ids': [[13959, 936, 12, 1437, 10, 10440, 12219, 2735, 1], [13959, 936, 12, 1437, 10, 3396, 254, 6037, 12920, 968, 9047, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[2735, 18, 4198, 5947, 1], [9047, 5947, 11590, 1]]})
{'input_ids': [[13959, 936, 12, 1437, 10, 10440, 12219, 2735, 1], [13959, 936, 12, 1437, 10, 3396, 254, 6037, 12920, 968, 9047, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[2735, 18, 4198, 5947, 1], [9047, 5947, 11590, 1]]}
--------------------------------------------------
Input Tokens  :  [13959, 936, 12, 1437, 10, 10440, 12219, 2735, 1]
Input IDs     :  translate human to machine: 12, Sep 2010</s>
--------------------------------------------------
Output Tokens :  [2735, 18, 4198, 5947, 1]
Output IDs    :  2010-09-12</s>


### Dataset 전체에 걸쳐 전처리하기

In [19]:
tokenized_datasets = dataset.map(
                        preprocess_function, 
                        batched=True,
                        remove_columns=dataset["train"].column_names,
                     )

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [21]:
train_dataset = tokenized_datasets["train"]
valid_dataset = tokenized_datasets["validation"]

## T5 준비 

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

tokenizer_id = MODEL_ID

# 8bit quantization 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=True,
)

# 모델 로딩
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

'\nfrom transformers import AutoModelForSeq2SeqLM\nmodel = AutoModelForSeq2SeqLM.from_pretrained(\n    MODEL_ID,\n    #device_map="auto",   # 모델에 따라 지원되는게 있고 아닌게 있습니다. (오래된 모델은 안됩니다.)\n    load_in_8bit=True,\n    trust_remote_code=True\n)\n\n# check https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb\nfrom peft import prepare_model_for_int8_training\nmodel = prepare_model_for_int8_training(model)\n'

### Model Structure 확인

In [24]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
              (k): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
              (v): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
              (o): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear8bitLt(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024

### Model Inference 수행해보기

In [25]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

번역에 맞는 몇 가지 입출력 전처리를 추가합니다.
1. Batch 내 입력, 출력의 최대 길이 조정
2. Input뿐만 아니라 Output레이블에도 Padding을 추가해줍니다.
3. Decoder input 에 start symbol에 해당하는 pad를 넣어줍니다.

In [26]:
sample_batch = [tokenized_datasets["train"][i] for i in range(1, 3)]

In [27]:
print( sample_batch[0] )
print( sample_batch[1] )

{'input_ids': [13959, 936, 12, 1437, 10, 3396, 254, 6037, 12920, 968, 9047, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [9047, 5947, 11590, 1]}
{'input_ids': [13959, 936, 12, 1437, 10, 505, 446, 7443, 5247, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [5247, 18, 5176, 18, 4018, 1]}


In [28]:
batch = data_collator(sample_batch)

In [29]:
batch.keys()

KeysView({'input_ids': tensor([[13959,   936,    12,  1437,    10,  3396,   254,  6037, 12920,   968,
          9047,     1],
        [13959,   936,    12,  1437,    10,   505,   446,  7443,  5247,     1,
             0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]), 'labels': tensor([[ 9047,  5947, 11590,     1,  -100,  -100],
        [ 5247,    18,  5176,    18,  4018,     1]]), 'decoder_input_ids': tensor([[    0,  9047,  5947, 11590,     1,     0],
        [    0,  5247,    18,  5176,    18,  4018]])})

In [30]:
print( batch['input_ids'] )

tensor([[13959,   936,    12,  1437,    10,  3396,   254,  6037, 12920,   968,
          9047,     1],
        [13959,   936,    12,  1437,    10,   505,   446,  7443,  5247,     1,
             0,     0]])


In [31]:
print( batch['attention_mask'] )

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [32]:
print( batch['labels'] )

tensor([[ 9047,  5947, 11590,     1,  -100,  -100],
        [ 5247,    18,  5176,    18,  4018,     1]])


In [33]:
print( batch['decoder_input_ids'] )

tensor([[    0,  9047,  5947, 11590,     1,     0],
        [    0,  5247,    18,  5176,    18,  4018]])


collator로 인해 2가지가 새로 변한 것을 알 수 있습니다.
- decoder_input_ids는 앞에 0 즉, pad symbol 이 추가되었습니다.
- labels 와 decoder_input_ids 를 살펴보면 하나씩 오른쪽으로 shift되었음을 확인할 수 있습니다.

In [38]:
#batch.to('cuda:0')
#outputs = model(**batch)

batch = {k: v.to('cuda:0') for k, v in batch.items()}
outputs = model(**batch)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [39]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [40]:
print(outputs.loss)
print(outputs.logits)

tensor(4.0820, device='cuda:0', dtype=torch.float16,
       grad_fn=<NllLossBackward0>)
tensor([[[-14.8281,  -7.7695, -10.2188,  ..., -36.8125, -36.9375, -36.8438],
         [-21.9531,  -0.7720,  -7.7930,  ..., -36.9062, -37.0000, -36.9375],
         [-26.1719,  -7.3789,  -8.1719,  ..., -37.0312, -37.1875, -37.1875],
         [-22.7344,   0.1759,  -6.7461,  ..., -36.3438, -36.4375, -36.3438],
         [-25.5469,  -1.1611,  -9.2109,  ..., -38.0625, -38.2500, -38.0938],
         [-10.9062,  -2.8242,  -8.3125,  ..., -35.7812, -35.9688, -35.7500]],

        [[-18.6562,  -8.5469, -11.0156,  ..., -36.5625, -36.6875, -36.7188],
         [-24.7031,  -1.7607,  -8.9375,  ..., -37.7500, -37.7812, -37.7812],
         [-24.0000,  -3.2949,  -6.4023,  ..., -32.5312, -32.5625, -32.5625],
         [-26.8750,  -7.3047,  -8.4062,  ..., -36.3438, -36.4062, -36.4375],
         [-31.0000,  -7.8281, -11.0234,  ..., -38.6562, -38.5625, -38.7188],
         [-27.1250,  -1.1875,  -8.8672,  ..., -39.1562, -39.250

### 예측 결과를 Symbol 로 바꾸기

In [41]:
import torch
best_output = torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() 
tokenizer.convert_ids_to_tokens( best_output )

['▁D', '</s>', '-14', '</s>', '</s>', '▁']

어떻게든 답은 나오지만, 제대로 된 결과가 아님을 알 수 있습니다. 훈련을 통해 제대로 된 결과를 얻어보죠.

## Fine-Tuning 수행하기

In [33]:
import wandb
wandb.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of

True

### Evaluation Metric 

In [42]:
import evaluate
metric = evaluate.load("sacrebleu")

### SacreBLEU
SacreBLEU는 자연어 처리(NLP) 분야에서 기계 번역의 품질을 평가하기 위해 널리 사용되는 도구입니다. BLEU(BiLingual Evaluation Understudy) 점수는 기계 번역의 출력과 사람이 번역한 참조 번역 사이의 유사도를 측정하는 방법 중 하나로, SacreBLEU는 이 BLEU 점수를 보다 일관되고 신뢰할 수 있는 방식으로 계산합니다.

In [43]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result


위의 compute_metrics 는 [Transformer 번역 예제](https://huggingface.co/course/chapter7/4?fw=pt#metrics)에서 가져왔습니다.

### LORA화 하기

In [47]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)


from peft import get_peft_model
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 't5-large' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [48]:
model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PeftModelForSeq2SeqLM(
      (base_model): LoraModel(
        (model): T5ForConditionalGeneration(
          (shared): Embedding(32128, 1024)
          (encoder): T5Stack(
            (embed_tokens): Embedding(32128, 1024)
            (block): ModuleList(
              (0): T5Block(
                (layer): ModuleList(
                  (0): T5LayerSelfAttention(
                    (SelfAttention): T5Attention(
                      (q): lora.Linear8bitLt(
                        (base_layer): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.1, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=1024, out_features=16, bias=False)
                        )
                        (lora_B): ModuleDict(
                     

### Huggingface 의 Trainer 활용하여 훈련하기

In [57]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    f"{MODEL_ID}-finetuned-human-to-machine",
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=2,             # 훈련 에폭 수
    per_device_train_batch_size=1024, # 디바이스 당 훈련 배치 크기
    
    ## ---- Learning Rate ----- ##
    #warmup_steps=500,                # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율
    learning_rate=2e-5,
    
    ## - Logging & Checkpoint - ##
    logging_steps=10,                # 로그를 기록할 스텝 간격
    save_steps=1000,                 # 체크포인트를 저장할 스텝 간격
    
    ## ---- Evaluations ----- ##
    predict_with_generate=True,
    eval_strategy="epoch",
)

### Trainer 설정하기

In [61]:
from transformers import Seq2SeqTrainer
# 1. 커스텀 Trainer 정의
class FixedSeq2SeqTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        kwargs.pop("num_items_in_batch", None)
        return super().compute_loss(model, inputs, return_outputs=return_outputs)

# 2. Trainer 생성 (클래스명만 변경)
trainer = FixedSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_10406/2477155207.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FixedSeq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = FixedSeq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [62]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [63]:
trainer.train()

TrainOutput(global_step=978, training_loss=0.6175288724996805, metrics={'train_runtime': 3911.2177, 'train_samples_per_second': 255.675, 'train_steps_per_second': 0.25, 'total_flos': 7.91821092028416e+16, 'train_loss': 0.6175288724996805, 'epoch': 2.0})

In [64]:
trainer.save_model("./results")

## 훈련된 모델 Inference

In [65]:
# 1) outputs 에 저장된 파일을 불러오고
# 2) 불러온 모델에 대해 다시 PEFT 시킨다.
pretrained_model_path = "./results"
lora_config = LoraConfig.from_pretrained(pretrained_model_path)
model = get_peft_model(model, lora_config)
model.cuda()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PeftModelForSeq2SeqLM(
      (base_model): LoraModel(
        (model): PeftModelForSeq2SeqLM(
          (base_model): LoraModel(
            (model): T5ForConditionalGeneration(
              (shared): Embedding(32128, 1024)
              (encoder): T5Stack(
                (embed_tokens): Embedding(32128, 1024)
                (block): ModuleList(
                  (0): T5Block(
                    (layer): ModuleList(
                      (0): T5LayerSelfAttention(
                        (SelfAttention): T5Attention(
                          (q): lora.Linear8bitLt(
                            (base_layer): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
                            (lora_dropout): ModuleDict(
                              (default): Dropout(p=0.1, inplace=False)
                            )
                            (lora_A): ModuleDict(
                              (default): Linear(i

In [66]:
input_text = ['translate human to machine: '+x for x in dataset['validation']['human'][:2] ]

In [67]:
input_text

['translate human to machine: 2/27/76',
 'translate human to machine: November 14, 1999']

In [68]:
inputs = tokenizer(input_text, 
                   return_tensors="pt", 
                   padding=True, 
                   max_length=max_token_length)

In [70]:
inputs = {k: v.to(model.device) for k, v in inputs.items()}
machines = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

machines.shape

torch.Size([2, 7])

In [71]:
[ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(m)) for m in machines
]

['<pad> 2/27/76</s>', '<pad> 14. November 1999</s></s></s>']